In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, confusion_matrix, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from scipy.ndimage import gaussian_filter1d
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
df_natural_gas = pd.read_csv("./datasets/Nat_Gas.csv")
df_natural_gas

,Dates,Prices
0,10/31/20,10.10
1,11/30/20,10.30
2,12/31/20,11.00
3,1/31/21,10.90
4,2/28/21,10.90
5,3/31/21,10.90
6,4/30/21,10.40
7,5/31/21,9.84
8,6/30/21,10.00
9,7/31/21,10.10


In [63]:
def pricing_the_contract(injection_dates_lst:[str], withdrawal_dates_lst:[str], purchasing_price_lst:[float], sold_price_lst:[float], injection_volume_lst:[float], withdraw_volume_lst:[float], storage_costs_per_month = 30, max_volume=2000):
    total_profit = 0
    stop = False
    if len(injection_dates_lst) == len(withdrawal_dates_lst) == len(purchasing_price_lst) == len(sold_price_lst) == len(injection_volume_lst) == len(withdraw_volume_lst) :
        for i in range(len(injection_dates_lst)):
            injection_dates = injection_dates_lst[i]
            withdrawal_dates = withdrawal_dates_lst[i]
            purchasing_price = purchasing_price_lst[i]
            sold_price = sold_price_lst[i]
            injection_volume = injection_volume_lst[i]
            withdraw_volume = withdraw_volume_lst[i]
            if not (isinstance(storage_costs_per_month, (int, float)) and storage_costs_per_month >= 0):
                raise ValueError("storage_costs_per_month must be a non-negative number")

            if not (isinstance(max_volume, (int, float)) and max_volume >= 0):
                raise ValueError("max_volume must be a non-negative number")

            if not (isinstance(purchasing_price, (int, float)) and purchasing_price >= 0):
                raise ValueError("purchasing_price must be a non-negative number")

            if not (isinstance(sold_price, (int, float)) and sold_price >= 0):
                raise ValueError("sold_price must be a non-negative number")

            if not (isinstance(injection_volume, (int, float)) and injection_volume >= 0):
                raise ValueError("injection_withdraw_volume must be a non-negative number")

            if not (isinstance(withdraw_volume, (int, float)) and withdraw_volume >= 0):
                raise ValueError("injection_withdraw_volume must be a non-negative number")

            if sum(injection_volume_lst) != sum(withdraw_volume_lst):
                print('All injection must also be withdrawn if and only if there exist equal volume of injected and withdrew of natural gas')
                stop = True
                break
            difference_in_unit_price = sold_price - purchasing_price
            total_profit -= difference_in_unit_price * injection_volume
            injection_dates = pd.to_datetime(injection_dates)
            withdrawal_dates = pd.to_datetime(withdrawal_dates)
            if injection_dates == withdrawal_dates:
                total_profit += 0
                continue
            else:
                if int((withdrawal_dates - injection_dates) / np.timedelta64(1, 'M')) < 1:
                    difference_in_months = 1
                else:
                    difference_in_months = int((withdrawal_dates - injection_dates) / np.timedelta64(1, 'M'))
            # print(difference_in_months)
            storage_cost = storage_costs_per_month * difference_in_months

            revenue = difference_in_unit_price * withdraw_volume
            intermediate_profit = revenue - storage_cost
            total_profit += intermediate_profit
            continue
        if not stop:
            return total_profit
    else:
        print('List length unmatched')



In [65]:
pricing_the_contract(injection_dates_lst=['10/31/20', '9/30/24'], withdrawal_dates_lst=['4/30/21', '9/30/24'], purchasing_price_lst=[10.10, 11.80], sold_price_lst=[10.40, 11.80], injection_volume_lst=[1000, 500], withdraw_volume_lst=[500, 1000])

-300.00000000000034